In [ ]:
from openai import OpenAI
from ipywidgets import widgets, Layout, Button, VBox, Textarea, HBox, HTML, Output, Image as ImageWidget
from IPython.display import display

# Basis-Layout-Optionen
base_layout = Layout(display='flex', justify_content='center', align_items='center')

# Basis-Layout-Optionen und Stil-Optionen für Widgets
button_layout = Layout(width='150px', height='40px', margin='5px 10px 5px 10px')
text_input_layout = Layout(flex='2 1 auto', min_height='auto', margin='10px 0 10px 0')
send_button_layout = Layout(flex='0 1 auto', height='auto', margin='10px 0 10px 0')
response_output_layout = Layout(width='60%', min_height='auto', overflow='auto', margin='10px 0 10px 0')

# CSS für individuelles Styling einschließlich Hintergrundbild
css_style = """
<style>
    .background-container { 
      background-image: url('bg.png');
      background-repeat: no-repeat;
      background-position: center center; 
      background-size: cover;
      margin: 0; 
      height: 100vh;
      width: 100vw;
      overflow: hidden;
    }
    .main-box {
      justify-content: center;
      align-items: center;
      display: flex;
      flex-direction: column;
      height: 100vh;
      width: 100vw;
    }
    .custom-textarea, .example-button, .send-button {
      background-color: #72B957; 
      color: white; 
      font-family: 'Barlow', sans-serif; 
      border-radius: 5px;
    }
    .example-button { background-color: #0B3653; }

    /* Stil für den image_container */
    .image-container {
      display: flex;
      justify-content: center; /* Zentriert den Inhalt horizontal */
      align-items: center; /* Zentriert den Inhalt vertikal */
      width: 500px; /* Breite des Containers auf die Breite des Bildes beschränken */
      height: 500px; /* Höhe des Containers auf die Höhe des Bildes beschränken */
    }
</style>
"""
display(HTML(css_style))

# Init OpenAI API
client = OpenAI()

# Ladeanzeige hinzufügen
loading_indicator = HTML(value="<i>Bild wird geladen...</i>")

# Funktion zum Senden der Anfrage und Anzeigen des Bildes
def generate_and_show_image(b):
    message = text_input.value

    # Ladeanzeige anzeigen
    image_container.clear_output()
    with image_container:
        display(loading_indicator)

    # Integration der prompt_image Funktion, um die Bild-URL zu erhalten
    image_url = prompt_image(message)

    # Ladeanzeige entfernen und Bild anzeigen
    image_container.clear_output()
    with image_container:
        display(ImageWidget.from_url(image_url, width=500, height=500))

def prompt_image(message:str, width:int=1024, height:int=1024, model='dall-e-3'):
    # OpenAI API-Anfrage und Rueckgabe der URL des generierten Bildes
    response = client.images.generate(
      prompt=message,
      model=model,
      n=1,
      size=f"{width}x{height}"
    )
    image_url = response.data[0].url
    
    return image_url


# Beispiel-Prompt-Btns
example_prompts = [
    Button(description="Sonnenuntergang", layout=button_layout),
    Button(description="Berglandschaft", layout=button_layout),
    Button(description="Stadtbild bei Nacht", layout=button_layout)
]

def set_prompt_example(b):
    example_texts = [
        "Ein Sonnenuntergang am Strand mit Palmen",
        "Eine ruhige Berglandschaft mit einem klaren See im Vordergrund",
        "Eine belebte Stadtszene bei Nacht mit leuchtenden Straßenlaternen"
    ]
    text_input.value = example_texts[example_prompts.index(b)]

for btn in example_prompts:
    btn.add_class('example-button')
    btn.on_click(set_prompt_example)

# Widgets und ihre Anordnung
text_input = Textarea(placeholder='Geben Sie hier Ihren Text ein', layout=text_input_layout)
text_input.add_class('custom-textarea')

send_button = Button(description="Senden", layout=send_button_layout)
send_button.add_class('send-button')
send_button.on_click(generate_and_show_image)

# `image_container` als Output-Widget
image_container = widgets.Output(layout=Layout(
    border='1px solid black'
))
image_container.add_class('image-container')  # Klasse zuweisen

# Anordnung der Widgets mit Hintergrundklasse
main_box = VBox([
    HBox(example_prompts, layout=base_layout),
    HTML(value="<b>Eingabe:</b>", style={'margin': '10px'}),
    HBox([text_input, send_button], layout=Layout(display='flex', flex_flow='row', align_items='stretch', width='60%')),
    HTML(value="<b>Bildausgabe:</b>", style={'margin': '10px'}),
    image_container
], layout=Layout(align_items='center', justify_content='center', width='100%', flex_flow='column', display='flex'))

main_box.add_class('main-box')
main_box.add_class('background-container')

display(main_box)
